- Find the oldest product
- Find the newest product
- Average product age, where age means how long the product has been in the system.
- List of other countries where products are sold too.
- Identify category of products and the compute:
   Number of products by category
    List containing names of products by category 
- Identify traces and compute:
    Number of products by trace
    List containing names of products by trace
- Data quality analysis on fields of interest (see appendix 1):
   Number of products with complete info.
    % of products without complete analysis per 100g
    % of products without additives info
    % of products without traces info
- Data profiling on fields of interest (see appendix 1):
    Stats on analysis per 100g fields

In [1]:
#Importing APIs
import pandas as pd
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window
from IPython.display import display, Markdown
from pyspark.sql import SQLContext


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sqlContext = SQLContext(spark)

In [134]:
#Load dataset
resto = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", "\t") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "false") \
                 .csv("noTT.csv")
tt = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("title.csv")

In [135]:
resto.show(3)
rest = resto.drop("C_185")
#.drop("C_173").drop("C_174").drop("C_175")
rest.show(1)
tt.show(1)
concate = tt.union(rest)

+----------+--------------------+-------------+----------+-------------------+----------+-------------------+--------------------+----+----------------+-----+---------+---------+----+------------------+------------------+--------------------+--------------------+--------------------+----+----+----+-------+-------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+-----------+----------+-------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+-----+----+----+----+--------------------+--------------------+----+----+----+----+----+----+----+----+----+-------------+-------+--------------------+--------------------+--------------------+----+--------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-----+------+----+----+---------------+----+----+----+----+----+----+----+----+--

In [136]:
concate.show(2)

+----+--------------------+-------------+----------+-------------------+---------------+----------------------+--------------------+------------------------+------------+--------+---------+--------------+--------------+------------------+------------------+-----------+---------------+-------------+-------+------------+----+--------------------+-------------------------+-------------+----------------+-------------+--------------+-------------------------+------+-----------+----+---------------+------+---------+--------------+------------+----------------+---------+--------------+------+-----------+---------+------------+----------------+-----------+--------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------------------+------------------------------------------+----------------+----------------+----------+-------------+-------------+------+-----------+-------------+-----------------

In [137]:
pdcont = concate.toPandas()

In [138]:
pd.set_option('display.max_columns', None)
pdcont.head(30)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,??,manufacturing_places,manufacturing_places_tags,labels,labels_tags,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,???,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_tags,traces,traces_tags,traces_en,serving_size,serving_quantity,additives_n,additives_tags,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,brand_owner,main_category,?61,?62,?63,?64,?65,?66,image_url,image_small_url,image_front_url,image_front_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,#NAME?81,#NAME?82,#NAME?83,#NAME?84,#NAME?85,#NAME?86,#NAME?87,#NAME?88,#NAME?89,#NAME?90,#NAME?91,#NAME?92,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,#NAME?96,#NAME?97,#NAME?98,omega-6-fat_100g,#NAME?100,#NAME?101,#NAME?102,#NAME?103,omega-9-fat_100g,#NAME?105,#NAME?106,#NAME?107,#NAME?108,#NAME?109,#NAME?110,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,#NAME?115,#NAME?116,#NAME?117,#NAME?118,#NAME?119,#NAME?120,starch_100g,polyols_100g,fiber_100g,#NAME?124,#NAME?125,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,17.0,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1591989744,2020-06-12 21:22:24,1609478763,2021-01-01 06:26:03,Vitória crackers,None,None,None,barquette,barquette,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Allemagne,en:germany,Germany,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",None,None,None,None,None,None,None,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,None,375.0,1569.0,None,7.0,3.0799999237061,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,70.099998474121,15.0,None,None,None,None,None,None,None,None,None,None,None,7.8000001907349,None,None,None,1.3999999761581,0.55999999046324,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [81]:
pdcont.describe()

,code,url,creator,created_t,last_modified_t,product_name,abbreviated_product_name,generic_name,quantity,packaging,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
count,76055,76055,76055,76055,76055,76055,76055,64107,1,12624,...,18,114,46,8,7,347,13,540,1,484
unique,76032,76055,2083,75673,75673,73061,73061,49471,1,10077,...,15,73,30,4,6,97,11,112,1,100
top,1.0,http://world-en.openfoodfacts.org/product/4056...,openfoodfacts-contributors,1561571298,2019-06-26 19:48:23,1607376555,2020-12-07 22:29:15,Kichererbsen,Bq 2 blancs plt panes s/at gaul,Natürliches Mineralwasser mit Kohlensäure vers...,...,9.9E-6,0.002,0.032,0.4,7.0,0.0,90.0,50.0,15.0,30.0
freq,3,1,14192,6,6,13,13,59,1,73,...,3,25,5,5,2,97,2,106,1,97


In [32]:
#pdfood = pd.read_csv("noT.csv", sep = "\t")

In [139]:
#checking the right position of the columns
concate.filter(col("ingredients_text") != "").select("ingredients_text").show(25)

+--------------------+
|    ingredients_text|
+--------------------+
|Zucker, Kakaomass...|
|Zucker,Palmöl, Ha...|
|Proteinmischung (...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|sugar, corn syrup...|
|sugar, corn syrup...|
|Natürliches Miner...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Enthält: 10 mg Me...|
|Tapioca Syrup, Ve...|
|         quinoa 100%|
|Sojaproteinisolat...|
|Heringsfilets (88...|
|Cucumbers, water,...|
|Composition pour ...|
|corn syrup, sugar...|
|water, soybean oi...|
|Branntweinessig, ...|
|Distilled Vinegar...|
|Branntweinessig, ...|
+--------------------+
only showing top 25 rows



Creator
Created_datetime
Last_modified_datetime
Product_name
Countries_en
Traces_en
Additives_tags
Main_category_en
Image_url
Quantity
Packaging_tags
Categories_en
Ingredients_text
Additives_en
Energy-kcal_100g
Fat_100g
Saturated-fat_100g
Sugars_100g
Salt_100g/sodium_100g

In [158]:
prepared_df = concate.select("Creator","created_t",
"Last_modified_t",
"Product_name",
"countries_en",
"traces_en",
"Additives_tags",
"Main_category",
"Image_url",
"Quantity",
"Packaging_tags",
"Categories",
"Ingredients_text",
"Additives_n",
"energy-kj_100g",
"Fat_100g",
"Saturated-fat_100g",
"Sugars_100g",
"Salt_100g")
#note: could not find additive, replaced with additive_n

In [ ]:
prepared_df.show(3)

In [141]:
prepared_df.filter(col("Salt_100g") != "").select("Salt_100g").count()

45460

# Subset data analysis in Spark

## Find the oldest product

In [142]:
prepared_df.select("created_t", "product_name").sort(col("created_t"),ascending = False).show(1)

+----------+--------------+
| created_t|  product_name|
+----------+--------------+
|1612662755|Sandwich Sauce|
+----------+--------------+
only showing top 1 row



## Find the newest product

In [105]:
prepared_df.select("created_t", "Product_name").sort(col("created_t"),ascending = True).show(1)

+----------+--------------------+
| created_t|        Product_name|
+----------+--------------------+
|1329035567|Milka Noisette En...|
+----------+--------------------+
only showing top 1 row



## Average product age, where age means how long the product has been in the system.

In [143]:
#convertion from unix format seconds to days to years = 3600*24*365
prepared_df.filter(col("created_t") > 0 ).agg((avg("created_t")/31536000).alias("average time (years)")).show()

+--------------------+
|average time (years)|
+--------------------+
|   49.15678858316572|
+--------------------+



## List of other countries where products are sold too.

In [155]:
prepared_df.select(explode(split(col("countries_en"), ",")).alias("indivcountry"))  \
            .filter((col("indivcountry") != "Allemagne") & (col("indivcountry") != "Germany") & (col("indivcountry") != "Deutschland") & ( col("indivcountry") != "en:DE") & ( col("indivcountry") != "en:de")) \
            .select("indivcountry").distinct() \
            .show(15)

+--------------------+
|        indivcountry|
+--------------------+
|  Middle-east-africa|
|       Côte d'Ivoire|
|          Luxemburgo|
|             Andorre|
|                Inde|
|Czech-republic-če...|
|     Greece-ελληνικά|
|              Russia|
|          nl:pologne|
|      Estados-unidos|
|      Hungary-magyar|
|      Romania-romană|
|      Hong-kong-粵語|
|       Malaysia-中文|
|               Suiza|
+--------------------+
only showing top 15 rows



## Identify traces and compute: Number of products by trace List containing names of products by trace

In [176]:
prepared_df.filter(col("traces_en") != 'null').groupby("traces_en") \
            .agg(count("Product_name").alias("product_total")) \
            .sort("product_total",ascending =False) \
            .show(40)

+--------------------+-------------+
|           traces_en|product_total|
+--------------------+-------------+
|                Nuts|         1319|
|        Nuts,Peanuts|          464|
|            Soybeans|          457|
|                Milk|          441|
|      Celery,Mustard|          437|
|        Sesame seeds|          321|
|         Gluten,Nuts|          246|
|              Celery|          245|
|              Gluten|          233|
|           Milk,Nuts|          223|
|       Nuts,Soybeans|          203|
|                Eggs|          177|
|             Mustard|          152|
|   Nuts,Sesame seeds|          147|
|   Eggs,Nuts,Peanuts|          127|
|Eggs,Gluten,Nuts,...|          121|
|       Eggs,Soybeans|          119|
|Sulphur dioxide a...|          117|
|Nuts,Sesame seeds...|          104|
|  Milk,Nuts,Soybeans|           99|
|Milk,Nuts,Sesame ...|           97|
|Milk,Nuts,Peanuts...|           93|
|Celery,Mustard,So...|           90|
|       Milk,Soybeans|           83|
|

## Data quality analysis on fields of interest (see appendix 1): Number of products with complete info. % of products without complete analysis per 100g % of products without additives info % of products without traces info

In [225]:
prepared_df.filter((col("Creator").isNotNull()) & 
                   (col("Last_modified_t").isNotNull()) &
                  (col("Product_name").isNotNull()) & 
                   (col("countries_en").isNotNull()) & 
                   (col("traces_en").isNotNull()) &
                   (col("Additives_tags").isNotNull()) & 
                   (col("Packaging_tags").isNotNull()) & 
                   (col("Categories").isNotNull()) & 
                   (col("Ingredients_text").isNotNull()) & 
                   (col("Energy-kcal_100g").isNotNull()) & 
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) ).count()


0

In [227]:
productsid = prepared_df.select(col("created_t")).count()
null100 = prepared_df.filter((col("Energy-kcal_100g").isNotNull()) & 
                   (col("Saturated-fat_100g").isNotNull()) & 
                   (col("Sugars_100g").isNotNull()) & 
                   (col("Salt_100g").isNotNull()) ).count()
(100 * null100 / productsid)

0.0

In [223]:
nulladd = prepared_df.filter( col("additives_tags").isNull()).count()
print("Ratio of null additive to all product")
(100 * nulladd / productsid)

ratio of null additive to all product


53.335086450594964

In [226]:
nulltrace = prepared_df.filter( col("traces_en").isNull()).count()
print("Ratio of null traces to all product")
(100 * nulltrace / productsid)

Ratio of null traces to all product


83.98790349089475

In [192]:
#dictanalysis = [prepared_df.filter(col(column).isNotNull()).count() for column in prepared_df.schema.names]

+-------+
|Creator|
+-------+
|kiliweb|
+-------+
only showing top 1 row

+----------+
| created_t|
+----------+
|1591989744|
+----------+
only showing top 1 row

+---------------+
|Last_modified_t|
+---------------+
|     1609478763|
+---------------+
only showing top 1 row

+----------------+
|    Product_name|
+----------------+
|Vitória crackers|
+----------------+
only showing top 1 row

+------------+
|countries_en|
+------------+
|     Germany|
+------------+
only showing top 1 row

+---------+
|traces_en|
+---------+
|     null|
+---------+
only showing top 1 row

+--------------+
|Additives_tags|
+--------------+
|          null|
+--------------+
only showing top 1 row

+-------------+
|Main_category|
+-------------+
|      unknown|
+-------------+
only showing top 1 row

+---------+
|Image_url|
+---------+
|     null|
+---------+
only showing top 1 row

+--------+
|Quantity|
+--------+
|    null|
+--------+
only showing top 1 row

+--------------+
|Packaging_tags|
+----------

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Data profiling on fields of interest (see appendix 1): Stats on analysis per 100g fields